# Sentinel Hub Process API services from within Python

Er wordt gebruikt gemaakt van de Sentinel Hub Process API om onze  satellietbeelden te verzamelen.
Er wordt een request request gestuured naar sentinel Hub en we kunne dan met get_data() de data krijgen en printen via Python

In [1]:
from python_scripts.connector import *
from python_scripts.helper import *

Succefully connected!


In [2]:
from eval_scripts.evalscript_true_color import *
from eval_scripts.evalscript_forest import *
from eval_scripts.evalscript_ndvi import *
from eval_scripts.evalscript_ndvi_values import *

In [3]:
from sentinelhub.geo_utils import *
from sentinelhub.constants import CRS
from sentinelhub.geometry import BBox
import math 
import matplotlib.pyplot as plt
from PIL import Image

lng = 87.118996
lat = 21.5439876
# lng, lat = to_wgs84(lng, lat, CRS.WGS84)

In [4]:
# degrees to radians
def deg2rad(degrees):
    return math.pi*degrees/180.0
# radians to degrees
def rad2deg(radians):
    return 180.0*radians/math.pi

# Semi-axes of WGS-84 geoidal reference
WGS84_a = 6378137.0  # Major semiaxis [m]
WGS84_b = 6356752.3  # Minor semiaxis [m]

# Earth radius at a given latitude, according to the WGS-84 ellipsoid [m]
def WGS84EarthRadius(lat):
    # http://en.wikipedia.org/wiki/Earth_radius
    An = WGS84_a*WGS84_a * math.cos(lat)
    Bn = WGS84_b*WGS84_b * math.sin(lat)
    Ad = WGS84_a * math.cos(lat)
    Bd = WGS84_b * math.sin(lat)
    return math.sqrt( (An*An + Bn*Bn)/(Ad*Ad + Bd*Bd) )

# Bounding box surrounding the point at given coordinates,
# assuming local approximation of Earth surface as a sphere
# of radius given by WGS84
def boundingBox(latitudeInDegrees, longitudeInDegrees, halfSideInKm):
    lat = deg2rad(latitudeInDegrees)
    lon = deg2rad(longitudeInDegrees)
    halfSide = 1000*halfSideInKm

    # Radius of Earth at given latitude
    radius = WGS84EarthRadius(lat)
    # Radius of the parallel at given latitude
    pradius = radius*math.cos(lat)

    latMin = lat - halfSide/radius
    latMax = lat + halfSide/radius
    lonMin = lon - halfSide/pradius
    lonMax = lon + halfSide/pradius

    return (rad2deg(latMin), rad2deg(lonMin), rad2deg(latMax), rad2deg(lonMax))

In [5]:
bb = BBox(boundingBox(lng, lat, 1), CRS.WGS84)
boundingBox(lng, lat, 1)

(87.10998270424972, 21.36466070925989, 87.12800929575026, 21.723314490740115)

In [6]:
# coords = [86.773796, 21.848934, 87.231445, 22.063497]
coords = [lng - 0.013, lat - 0.013, lng + 0.013, lat + 0.013]

no_clouds = { 
    "dataFilter": { 
        "maxCloudCoverage": 0
    } 
}

In [7]:
coords_Amazon_Rainforest =[[-61.64304123722222, -7.1977762330555555, -61.59664948444445, -7.151749156666667],
[-47.276085125277774, -5.1459865675000005, -47.229872031388894, -5.099955920555555],
[-72.68058219944444, -7.546572816111111, -72.63415454916667, -7.500546328333333],
[-54.907312499999996, -16.483174878333333, -54.8593125, -16.437143912222222],
[-56.41083402305556, -15.082882561666667, -56.36315190388889, -15.036849999444444],
[-56.45938240527778, -13.673990039444444, -56.411989040277774, -13.62795547],
[-55.1953125, -12.487397424166666, -55.14817974277778, -12.441361218611112],
[-58.31184397277777, -14.310413613611113, -58.26431294555555, -14.2643830875],
[-57.82104207194445, -9.679239864444444, -57.774349465, -9.633204256944445],
[-57.51602668138889, -9.841712924444446, -57.46928557527778, -9.795677582777778],
[-51.494502314722226, -6.24193727, -51.44820601861111, -6.19590667],
[-51.1439817775, 0.029311676666666668, -51.09794597194445, 0.07534719333333334],
[-44.55610613305555, -4.537154218055555, -44.50994044944444, -4.491124064166667],
[-51.14374599861111, -2.903435306388889, -51.097651248333335, -2.857403249722222],
[-52.64127957722222, -8.591274486666666, -52.59473186583334, -8.545247906111111],
[-54.09449481027778, -3.8356628486111113, -54.04836462083333, -3.7896283052777777],
[-46.360107985, -4.140418458611111, -46.31396597027778, -4.094391151388889],
[-56.46292321888889, -5.1920172141666665, -56.416710125277774, -5.1459865675000005],
[-56.15536277611111, -13.516114908611112, -56.10804416416667, -13.470086673611112],
[-54.556744950833334, -15.669065434444445, -54.50891077611111, -15.623036831666667],
[-64.1517942775, -0.44362786111111113, -64.10575847194444, -0.39759407750000003],
[-68.6720579625, -10.717961193611112, -68.62521955499999, -10.67193133111111],
[-47.88742315583333, -3.3925968516666667, -47.841316598333336, -3.3465685875],
[-53.909914925555555, -4.186445765555556, -53.86377291083333, -4.140418458611111],
[-59.71893239305556, -9.633204256944445, -59.672239786111106, -9.587168649444445],
[-60.46875, -11.454581911666667, -60.42180164305555, -11.408551905277777],
[-47.414771816388885, -4.4450939102777784, -47.36860613305556, -4.399063756666667],
[-52.94571418138889, -10.025854291388889, -52.89897307527778, -9.979818949722222],
[-48.071207199444444, -7.592599303611111, -48.02477954916667, -7.546572816111111],
[-49.99615893777778, -5.1459865675000005, -49.94994584416666, -5.099955920555555]]

# Amazon_1279.tiff_29.tiff
# Amazon_124.tiff_36.tiff
# Amazon_1712.tiff_26.tiff
# Amazon_727.tiff_30.tiff
# Amazon_822.tiff_20.tiff
# Amazon_819.tiff_11.tiff
# Amazon_720.tiff_40.tiff
# Amazon_970.tiff_1.tiff
# Amazon_954.tiff_36.tiff
# Amazon_898.tiff_3.tiff
# Amazon_425.tiff_44.tiff
# Amazon_404.tiff_52.tiff
# Amazon_5.tiff_50.tiff
# Amazon_390.tiff_12.tiff
# Amazon_505.tiff_26.tiff
# Amazon_639.tiff_49.tiff
# Amazon_58.tiff_41.tiff
# Amazon_832.tiff_43.tiff
# Amazon_801.tiff_42.tiff
# Amazon_692.tiff_6.tiff
# Amazon_1377.tiff_12.tiff
# Amazon_1561.tiff_37.tiff
# Amazon_170.tiff_38.tiff
# Amazon_640.tiff_53.tiff
# Amazon_1110.tiff_25.tiff
# Amazon_1154.tiff_40.tiff
# Amazon_122.tiff_33.tiff
# Amazon_511.tiff_35.tiff
# Amazon_181.tiff_34.tiff
# Amazon_323.tiff_38.tiff

In [ ]:
for i in coords_Amazon_Rainforest[:10]:
    img_true = simple_request(evalscript_true_color, i, config, False, other_args=no_clouds)
    plot_image(img_true)
    print(i)

In [ ]:
for bbox in coords_Amazon_Rainforest[:10]:
    img_ndvi = simple_request(evalscript_ndvi_values, bbox, config, False, other_args=no_clouds)
    min_ndvi = 0.75
    ndvi_copy = img_ndvi.copy()
    labels = np.where(ndvi_copy > min_ndvi, 255, 0)
    img = Image.fromarray(labels)
    img.show()

In [ ]:
img_true = simple_request(evalscript_true_color, coords_, config, False, other_args=no_clouds)
plot_image(img_true)

In [ ]:
img_ndvi = simple_request(evalscript_ndvi_values, coords_, config, False, other_args=no_clouds)
plot_image(img_ndvi)

In [ ]:
min_ndvi = 0.8
ndvi_copy = img_ndvi.copy()
labels = np.where(ndvi_copy > min_ndvi, 255, 0)

In [ ]:
from PIL import Image
img = Image.fromarray(labels)
img.show()

In [ ]:
img_true = simple_request(evalscript_true_color, coords, config, False, other_args=no_clouds)
plot_image(img_true)

In [ ]:
img_true_ = simple_request(evalscript_true_color, bb, config, False, other_args=no_clouds)
plot_image(img_true_)

In [ ]:
# img_forestry = simple_request(evalscript_forest, coords, config, False, other_args=no_clouds)
# plot_image(img_forestry, 2/255)

In [ ]:
img_nvdi = simple_request(evalscript_ndvi, bb, config, False, other_args=no_clouds)
plot_image(img_nvdi)

## Get images form a specific timestamp

In [ ]:
img_true_ = sentinel_request(evalscript_true_color, coords,('2020-02-05', '2020-04-25'), config, False, other_args=no_clouds)
plot_image(img_true_)

In [ ]:
img_ndvi_ = sentinel_request(evalscript_ndvi, coords,('2020-02-05', '2020-04-25'), config, False, other_args=no_clouds)
plot_image(img_ndvi_)